In [6]:
import urllib.request
from bs4 import BeautifulSoup
import bs4

In [7]:
import csv

In [8]:
import re

In [9]:
def downloaden(link: str) -> (str):
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    req = urllib.request.Request(link, headers={'User-Agent': user_agent})
    with urllib.request.urlopen(req) as resp:
        html = resp.read()
    return html

In [10]:
def get_lower_cmts(root_link):
    html = downloaden(root_link)
    #html = 
    soup = BeautifulSoup(html, 'html.parser')
    option = soup.new_tag('option')
    buttons = soup.find_all('option')
    output = {}
    for button in buttons[1:]:
        raw = str(button)
        comitee = re.search('>(.+?)<', raw).group(1)
        link = re.search('value="(.+?)"', raw).group(1)
        link = re.sub('&amp;', '&', link)
        comitee = re.sub('^\d+ ', '', comitee)
        output[comitee] = link
    return output

In [11]:
def get_table(uik_link):
    table_dict = {}
    for sdict in ['meta', 'absolute', 'percentage']:
        table_dict[sdict] = {}
    html = downloaden(uik_link)
    soup = BeautifulSoup(html, 'html.parser')
    raw_table = max(soup.find_all('table'), key=len)
    for row in raw_table.find_all('tr'):
        rcls = list(row.strings)
        cells = [i.strip() for i in rcls[2:]]
        if len(cells) == 5 and not cells[3]:
            table_dict['meta'][cells[0]] = cells[1]
        elif len(cells) == 5 and cells[3]:
            table_dict['absolute'][cells[0]] = cells[1]
            table_dict['percentage'][cells[0]] = cells[3][:-1]
    return table_dict

In [31]:
def write_a_table(tt_dict, fname, n=0):
    with open(fname, 'w', newline='') as csvfile:
        flkey = list(tt_dict.keys())[n]
        slkey = list(tt_dict[flkey].keys())[0]
        headers = list(tt_dict[flkey][slkey].keys())
        #print(fieldnames)
        fieldnames = ["ТИК", "УИК"]
        fieldnames.extend(headers)
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writeheader()
        for district in tt_dict:
            uiks = tt_dict[district]
            for uik in uiks:
                row = uiks[uik]
                row['ТИК'] = district
                row['УИК'] = re.sub('\D', '', uik)
                writer.writerow(row)
    

In [60]:
def update_a_table(tt_dict, fname):
    with open(fname, 'a') as csvfile:
        for district in tt_dict:
            uiks = tt_dict[district]
            if uiks:
                for uik in uiks:
                    row = [district, re.sub('\D', '', uik)]
                    for key in uiks[uik].keys():
                        row.append(uiks[uik][key])
                    csvfile.write(';'.join(row))
                    csvfile.write('\n')
                    

In [51]:
update_a_table(tt_meta, '%smeta.csv' % (fname))
update_a_table(tt_absolute, '%sabsolute.csv' % (fname))
update_a_table(tt_per, '%spercentage.csv' % (fname))

In [59]:
ustart

3439

In [55]:
#meta = 13
#per and abs = 9
len(re.findall(';', "район Чертаново Северное;2042;1.33"))

2

In [56]:
def fix_it(fname, n):
    with open(fname, 'r') as of:
        rows = of.read().split('\n')
    fixed = []
    for row in rows:
        if len(re.findall(';', row)) == n:
            fixed.append(row)
    nfname = 'fixed_' + fname
    with open(nfname, 'w') as nf:
        nf.write('\n'.join(fixed))

In [57]:
fix_it('president2018meta.csv', 13)
fix_it('president2018absolute.csv', 9)
fix_it('president2018percentage.csv', 9)

In [1]:
#link = 'http://www.moscow_city.vybory.izbirkom.ru/region/region/moscow_city?action=show&root=1&tvd=27720001368293&vrn=27720001368289&region=77&global=null&sub_region=77&prver=0&pronetvd=null&vibid=27720001368293&type=234'

def mayorelections(link, fname):
    districts = get_lower_cmts(link)
    tree = {}
    for d in districts:
        uiks = get_lower_cmts(districts[d])
        tree[d] = uiks
        
    tt_meta = {}
    tt_absolute = {}
    tt_per = {}

    for d_name in tree:
        district = tree[d_name]
        tt_meta[d_name] = {}
        tt_absolute[d_name] = {}
        tt_per[d_name] = {}
        for u_name in district:
            u_link = district[u_name]
            u_info = get_table(u_link)
            tt_meta[d_name][u_name] = u_info['meta']
            tt_absolute[d_name][u_name] = u_info['absolute']
            tt_per[d_name][u_name] = u_info['percentage']
            
    write_a_table(tt_meta, '%smeta.csv' % (fname))
    write_a_table(tt_absolute, '%sabsolute.csv' % (fname))
    write_a_table(tt_per, '%spercentage.csv' % (fname))

IndentationError: unexpected unindent (<ipython-input-1-ddc2bf271c0b>, line 27)

In [61]:
#link = 'http://www.moscow_city.vybory.izbirkom.ru/region/region/moscow_city?action=show&root=1&tvd=27720002197406&vrn=27720002197402&region=77&global=null&sub_region=77&prver=0&pronetvd=null&vibid=27720002197406&type=234'
#fname = 'mayor2018'

link = 'http://www.moscow_city.vybory.izbirkom.ru/region/region/moscow_city?action=show&root=1000074&tvd=100100084849200&vrn=100100084849062&region=77&global=true&sub_region=77&prver=0&pronetvd=null&vibid=100100084849200&type=226'
fname = 'fixed_president2018'

#dcount = 0
#districts = get_lower_cmts(link)
#tree = {}
#for d in districts:
#    dcount += 1
#    if dcount % 20 == 0:
#        print('districts done: %s' % (str(dcount)))
#    uiks = get_lower_cmts(districts[d])
#    tree[d] = uiks
    
#print('link tree finished')
    
tt_meta = {}
tt_absolute = {}
tt_per = {}

dcount = 0
ucount = 0
for d_name in tree:
    dcount += 1
    if dcount % 20 == 0:
        print('districts done: %s' % (str(dcount)))
    district = tree[d_name]
    tt_meta[d_name] = {}
    tt_absolute[d_name] = {}
    tt_per[d_name] = {}
    for u_name in district:
        u_link = district[u_name]
        ucount += 1
        if ucount >= ustart:           
            u_info = get_table(u_link)
            if ucount % 50 == 0:
                print('uiks done: %s' % (str(ucount)))
            tt_meta[d_name][u_name] = u_info['meta']
            tt_absolute[d_name][u_name] = u_info['absolute']
            tt_per[d_name][u_name] = u_info['percentage']
            
update_a_table(tt_meta, '%smeta.csv' % (fname))
update_a_table(tt_absolute, '%sabsolute.csv' % (fname))
update_a_table(tt_per, '%spercentage.csv' % (fname))

districts done: 20
districts done: 40
districts done: 60
districts done: 80
districts done: 100
districts done: 120
uiks done: 3450
uiks done: 3500
uiks done: 3550
uiks done: 3600


In [36]:
update_a_table(tt_meta, '%smeta.csv' % (fname))
update_a_table(tt_absolute, '%sabsolute.csv' % (fname))
update_a_table(tt_per, '%spercentage.csv' % (fname))

In [47]:
tt_absolute

{'район Богородское': {},
 'район Вешняки ': {},
 'район Восточное Измайлово ': {},
 'район Гольяново ': {},
 'район Ивановское ': {},
 'район Измайлово ': {},
 'район Косино-Ухтомский ': {},
 'район Метрогородок ': {},
 'район Новокосино ': {},
 'район Новогиреево ': {},
 'район Перово': {},
 'район Восточный': {},
 'район Преображенское': {},
 'район Северное Измайлово': {},
 'район Соколиная гора': {},
 'район Сокольники': {},
 'район Внуково': {},
 'район Дорогомилово': {},
 'район Крылатское': {},
 'район Кунцево': {},
 'Можайский район': {},
 'район Ново-Переделкино': {},
 'район Очаково-Матвеевское': {},
 'район Проспект Вернадского': {},
 'район Раменки': {},
 'район Солнцево': {},
 'район Тропарево-Никулино': {},
 'район Филевский парк': {},
 'район Фили-Давыдково': {},
 'район Силино': {},
 'район Крюково': {},
 'район Старое Крюково': {},
 'район Матушкино': {},
 'район Аэропорт': {},
 'район Беговой ': {},
 'Бескудниковский район ': {},
 'район Савелки': {},
 'Войковский ра

In [30]:
write_a_table(tt_meta, '%smeta.csv' % (fname), n=91)
write_a_table(tt_absolute, '%sabsolute.csv' % (fname), n=91)
write_a_table(tt_per, '%spercentage.csv' % (fname), n=91)

TypeError: write_a_table() got an unexpected keyword argument 'n'

In [ ]:
write_a_table(tt_meta, 'mayor2013meta.csv')
write_a_table(tt_absolute, 'mayor2013absolute.csv')
write_a_table(tt_per, 'mayor2013percentage.csv')

In [34]:
for key in tt_absolute["район Коньково"]["УИК №2233"].keys():
    print(key)

Бабурин Сергей Николаевич
Грудинин Павел Николаевич
Жириновский Владимир Вольфович
Путин Владимир Владимирович
Собчак Ксения Анатольевна
Сурайкин Максим Александрович
Титов Борис Юрьевич
Явлинский Григорий Алексеевич


In [ ]:
sample = get_table('http://www.moscow_city.vybory.izbirkom.ru/region/moscow_city?action=show&root=177413003&tvd=27720001368316&vrn=27720001368289&prver=0&pronetvd=null&region=77&sub_region=77&type=234&vibid=27720001368316')
for k in sample:
    print(k)
    for r in sample[k]:
        print('\t%s:\t%s' % (r, sample[k][r]))

In [80]:
def compare_uik(fname1, fname2, nfname1, nfname2):
    def opening(fname):
        with open(fname, 'r') as f:
            table = f.read().split('\n')[1:-1]
            rows = [row.split(';') for row in table]
        return rows
    rows1 = opening(fname1)
    rows2 = opening(fname2)
    def matching(rows):
        return {row[1]: row[0].strip() for row in rows}
    uiks1 = matching(rows1)
    uiks2 = matching(rows2) 
    print(len(uiks1) == len(rows1))
    print(len(uiks2) == len(rows2))
    f1only = [u1 for u1 in uiks1 if u1 not in uiks2]
    f2only = [u2 for u2 in uiks2 if u2 not in uiks1]
    if f1only:
        with open(nfname1, 'w') as nf1:
            for key in f1only:
                nf1.write('%s,%s\n' % (key, uiks1[key]))
    if f2only:
        with open(nfname2, 'w') as nf2:
            for key in f2only:
                nf2.write('%s,%s\n' % (key, uiks2[key]))
    

In [81]:
compare_uik('mayor2018percentage.csv', 'president2018.csv',
           'mayor2018only.csv', 'president2018only.csv')

True
True
